In [60]:
%load_ext tensorboard
import os
os.environ["TF_CPP_MIN_VLOG_LEVEL"] = "2"
import tensorflow as tf
import random
import numpy as np
import pandas as pd
import tensorrt as trt
from typing import Sequence, Optional, Tuple

print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
print("TensorRt:", trt.__version__)

RANDOM_STATE = 42

random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# Tensorflow uniform random generator
g1 = tf.random.Generator.from_seed(RANDOM_STATE)

TARGET = "rucwar"
MIN_PER_CLASS = 50
OUTER_SPLITS = 2
INNER_SPLITS = 2
BATCH_SIZE = 32

SAMPLE_RATE = 16000
MAX_DURATION = 10 # seconds
NEG_POS_RATIO = 3
TRAIN_NEG_RATIO = 2
VAL_NEG_RATIO = 2
FILL_TYPE = "pad" # pad | tile
N_MELS = 128
N_FRAMES = 157
FRAME_OVERLAP = int(np.ceil(N_FRAMES * 0.5))

SECONDS = 5
BYTES_PER_SECOND = SECONDS * SAMPLE_RATE
INPUT_SHAPE = (N_MELS, N_FRAMES, 1)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
TF: 2.19.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorRt: 10.13.2.6


In [61]:
from sklearn.model_selection import StratifiedGroupKFold
import os
import librosa

# birdclef sanity checks
def sanity_birdlcef(df: pd.DataFrame, target: str):
    # File integrity
    for r in df.itertuples():
        if os.path.isfile(r.path) is not True: # type:ignore
            print(f"[ERROR]: {r.path} is not valid!")
    
    # Dupicates
    dups = df['path'].duplicated().sum()  # or 'filename'
    print("Duplicate file rows:", dups)
    
    # Count summary
    print(f"Total Positives: {df[df['primary_label'] == target].shape[0]}")
    print(f"Total Negatives: {df[df['primary_label'] != target].shape[0]}")
    print(f"Avg samples per Class: {df['primary_label'].value_counts().mean():.0f}")



# Different functions for each dataset
def load_birdclef(audio_root, path, target, min_per_class = MIN_PER_CLASS):
    df = pd.read_csv(path)
    
    df["path"] = audio_root + "/" + df["primary_label"] + "/" + df["filename"]
    
    # Optional: drop rare classes (keeps CV stable)
    if min_per_class > 1:
        keep_labels = df["primary_label"].value_counts()
        keep_labels = keep_labels[keep_labels >= min_per_class].index
        df = df[df["primary_label"].isin(keep_labels)].reset_index(drop=True)
    
    # Perform sanity checks
    sanity_birdlcef(df, target)
    
    # Group based on auther + time te prevent straddeling
    df["group_key"] = df["author"] + df["time"]
    
    return df
    

os.chdir("/home/joris/Thesis/new_attempt")

birdclef_df = load_birdclef("datasets/birdclef_2021/train_short_audio", "datasets/birdclef_2021/train_metadata.csv", target=TARGET)


Duplicate file rows: 0
Total Positives: 154
Total Negatives: 61815
Avg samples per Class: 168


In [62]:
from sklearn.model_selection import StratifiedGroupKFold
import numpy as np
import pandas as pd

def make_nested_cv_splits(
    df: pd.DataFrame,
    target: str = TARGET,
    outer_splits: int = OUTER_SPLITS,
    inner_splits: int = INNER_SPLITS,
    random_state: int = RANDOM_STATE,
):
    """
    Build nested CV folds for binary BirdCLEF: target species = 1, others = 0.
    Uses StratifiedGroupKFold for both outer and inner splits to avoid leakage across groups.
    """
    
    # Binary labels for stratification
    y_all = (df["primary_label"] == target).astype(int).values
    groups_all = df["group_key"].astype(str).fillna("NA").values
    idx_all = np.arange(len(df))

    outer_kf = StratifiedGroupKFold(n_splits=outer_splits, shuffle=True, random_state=random_state)

    nested = []
    for k, (outer_tr_idx, outer_te_idx) in enumerate(outer_kf.split(idx_all, y=y_all, groups=groups_all), start=1):
        # Outer train/val pool and test set
        trval_idx = idx_all[outer_tr_idx]
        test_idx  = idx_all[outer_te_idx]

        y_trval   = y_all[outer_tr_idx]
        groups_trval = groups_all[outer_tr_idx]

        # Inner CV on the outer train/val pool
        inner_kf = StratifiedGroupKFold(n_splits=inner_splits, shuffle=True, random_state=random_state)
        inner_folds = []
        for j, (inner_tr_rel, inner_va_rel) in enumerate(inner_kf.split(trval_idx, y=y_trval, groups=groups_trval), start=1):
            # Map relative indices back to global indices
            inner_tr_idx = trval_idx[inner_tr_rel]
            inner_va_idx = trval_idx[inner_va_rel]

            inner_folds.append({
                "inner_fold": j,
                "inner_train_idx": inner_tr_idx,
                "inner_val_idx":   inner_va_idx,
            })

        nested.append({
            "outer_fold": k,
            "outer_train_idx": trval_idx,
            "outer_test_idx":  test_idx,
            "inner_folds": inner_folds,
            "train_pos_ratio": float(y_all[outer_tr_idx].mean()), # type: ignore
            "test_pos_ratio":  float(y_all[outer_te_idx].mean()), # type: ignore
        })

    return nested

    
    
cross_validation_sets = make_nested_cv_splits(birdclef_df)

In [ ]:
# Augments
def aug_gaussian_noise_tf(audio, snr_db):
    """
    Add Gaussian noise to a waveform at a given SNR (dB).
    
    Args:
        audio (tf.Tensor): 1D or 2D waveform tensor (e.g. [time] or [batch, time]).
        snr_db (float): Target signal-to-noise ratio in dB.
    
    Returns:
        tf.Tensor: Noisy waveform with the specified SNR.
    """
    # Calculate RMS of the signal
    rms_signal = tf.sqrt(tf.reduce_mean(tf.square(audio), axis=-1, keepdims=True))

    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_db / 20.0)

    # Desired noise RMS
    rms_noise = rms_signal / snr_linear

    # Generate Gaussian noise
    noise = g1.normal(shape=tf.shape(audio), dtype=tf.float32)

    # Normalize noise to unit RMS
    rms_noise_current = tf.sqrt(tf.reduce_mean(tf.square(noise), axis=-1, keepdims=True))
    noise = noise * (rms_noise / (rms_noise_current + 1e-8))

    return audio + noise

def aug_loudness_norm_tf(audio, target_db=-20.0, eps=1e-8):
    rms = tf.sqrt(tf.reduce_mean(tf.square(audio), axis=-1, keepdims=True) + eps)
    rms_db = 20.0 * tf.math.log(rms + eps) / tf.math.log(10.0)
    gain = 10.0 ** ((target_db - rms_db) / 20.0)
    return audio * gain

def aug_specaugment_tf(spec, max_freq_masks=2, max_time_masks=2,
                       max_freq_width=16, max_time_width=32):
    spec = tf.identity(spec)
    def _mask_freq(s):
        f = tf.shape(s)[-2]; t = tf.shape(s)[-1]
        w = g1.uniform((), 0, max_freq_width+1, dtype=tf.int32)
        start = g1.uniform((), 0, tf.maximum(f - w, 1), dtype=tf.int32)
        mask = tf.concat([tf.ones([start, t]), tf.zeros([w, t]), tf.ones([f - start - w, t])], axis=0)
        return s * mask
    def _mask_time(s):
        f = tf.shape(s)[-2]; t = tf.shape(s)[-1]
        w = g1.uniform((), 0, max_time_width+1, dtype=tf.int32)
        start = g1.uniform((), 0, tf.maximum(t - w, 1), dtype=tf.int32)
        mask = tf.concat([tf.ones([f, start]), tf.zeros([f, w]), tf.ones([f, t - start - w])], axis=1)
        return s * mask
    for _ in range(2):  # loop vars must be static; use fixed counts
        spec = _mask_freq(spec)
        spec = _mask_time(spec)
    return spec

In [ ]:
def _to_onehot(y, num_classes):
    # y can be [B] (int) or [B, 1] (int)
    if y.dtype.is_integer:
        y = tf.reshape(y, [-1])
        y = tf.one_hot(y, depth=num_classes, dtype=tf.float32)
    else:
        # already one-hot or float
        y = tf.cast(y, tf.float32)
    return y

def mixup_batch(x, y, alpha=0.2, num_classes=2):
    """
    x: [B, ...]   (your mel: [B, N_MELS, N_FRAMES, 1])
    y: [B] int, [B,1] int, or one-hot [B, C] / float labels
    returns (x_mix, y_mix) with the same shapes as input (labels -> one-hot/float)
    """
    B = tf.shape(x)[0]

    # Sample lambda ~ Beta(alpha, alpha)
    # Use two gammas trick (stable on TPU/GPU)
    gamma1 = g1.gamma([], alpha, 1.0)
    gamma2 = g1.gamma([], alpha, 1.0)
    lam = gamma1 / (gamma1 + gamma2)  # scalar

    # Permute the batch
    idx = g1.uniform([B], minval=0, maxval=B, dtype=tf.int32)
    x2 = tf.gather(x, idx)
    y2 = tf.gather(y, idx)

    # Ensure y is float one-hot (or float for binary)
    y  = _to_onehot(y,  num_classes)
    y2 = _to_onehot(y2, num_classes)

    # Mix
    lam_x = tf.cast(lam, x.dtype)
    x_mix = lam_x * x + (1.0 - lam_x) * x2

    lam_y = tf.cast(lam, tf.float32)
    y_mix = lam_y * y + (1.0 - lam_y) * y2  # soft labels

    return x_mix, y_mix

def maybe_mixup(x, y, p=0.5, alpha=0.2, num_classes=2):
    """Apply mixup to a batch with probability p."""
    u = g1.uniform(())
    return tf.cond(
        u < p,
        lambda: mixup_batch(x, y, alpha=alpha, num_classes=num_classes),
        lambda: (x, _to_onehot(y, num_classes) if y.dtype.is_integer else tf.cast(y, tf.float32))
    )


In [64]:
import subprocess

def load_ogg_ffmpeg(path, sr=SAMPLE_RATE):
    path = path.decode("utf-8")
    cmd = [
        "ffmpeg", "-i", path, "-f", "f32le",
        "-ac", "1", "-ar", str(sr), "pipe:1", "-loglevel", "quiet"
    ]
    out = subprocess.check_output(cmd)
    audio = np.frombuffer(out, np.float32)
    return audio

def load_ogg_librosa(path, sr=SAMPLE_RATE):
    path = path.decode("utf-8")
    y, sr = librosa.load(path, sr=SAMPLE_RATE)
    return y

from librosa._typing import _STFTPad

def generate_mel_spectrogram(audio, sr = SAMPLE_RATE, n_fft=1024, n_mels=128, hop_length=512, win_length=None, 
                       window:str='hann', center=True, pad_mode:_STFTPad='constant', power=2.0, fmin=200, 
                       fmax=8000, norm='slaney'):
    # Generate mel spectrogram
    spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, n_mels=n_mels, hop_length=hop_length, win_length=win_length, window=window, center=center, pad_mode=pad_mode, power=power, fmin=fmin, fmax=fmax, norm=norm)
    # Convert to dB
    spec = librosa.power_to_db(spec, ref=np.max)
    return spec

def apply_with_prob(x, p, aug_fn):
    """aug_fn must be a zero-arg callable returning a tensor like x."""
    u = g1.uniform((), 0.0, 1.0)
    return tf.cond(u < p, true_fn=aug_fn, false_fn=lambda: tf.identity(x))

def audio_pipeline(filename: Sequence[str], augment=False, p_loud=0.5, p_gaus=0.5, p_spec=0.8, loud_range=(-26.0, -18.0), gaus_range=(5, 20)):
    # Load audio file as tensor 
    audio_file = tf.numpy_function(load_ogg_librosa, [filename, SAMPLE_RATE], tf.float32)
    
    # Remove last dimension
    waveform = audio_file[:SAMPLE_RATE * MAX_DURATION] # type: ignore
    
    processed = waveform[:SAMPLE_RATE * SECONDS]
     
    if augment:
        # Loudness Normalization
        targetDbfs = g1.uniform((), loud_range[0], loud_range[1])
        processed = apply_with_prob(processed, p_loud, lambda: aug_loudness_norm_tf(processed, targetDbfs))
        
        # Add Gaussian noise    
        gaussian_snr = g1.uniform([], gaus_range[0], gaus_range[1])
        processed = apply_with_prob(processed, p_gaus, lambda: aug_gaussian_noise_tf(processed, gaussian_snr))
        
    
    n = tf.shape(processed)[0]
    # If shorter than desired, pad or tile
    if FILL_TYPE == "pad":
        pad = tf.maximum(0, SAMPLE_RATE * SECONDS - n)
        processed = tf.pad(processed, paddings=[[0, pad]])
    else:  # tile
        repeats = tf.maximum(
            1,
            tf.cast(
                tf.math.ceil((SAMPLE_RATE * SECONDS) / tf.cast(n, tf.float32)), tf.int32
            ),
        )
        processed = tf.tile(processed, [repeats])
        processed = processed[: SAMPLE_RATE * SECONDS]
    
    # Band filter
    from scipy import signal
    b, a = signal.butter(4, [200, 7999], fs=SAMPLE_RATE, btype='band')
    band_filter = tf.py_function(signal.lfilter, [b, a, processed], Tout=tf.float32, name="Filter")

    db_mel_spectrogram = tf.numpy_function(
        generate_mel_spectrogram, [band_filter, SAMPLE_RATE], Tout=tf.float32
    )

    db_mel_spectrogram = tf.ensure_shape(db_mel_spectrogram, shape=(N_MELS, N_FRAMES))

    
    return db_mel_spectrogram

In [65]:
import tensorflow.keras as keras

# --------- Simple binary CNN ----------
def build_binary_cnn(
    input_shape=(128, 64, 1),
    lr=1e-3,
    l2=1e-4,
    dropout=0.25,
    gamma=2.0,
    alpha=0.25,
):
    """
    Binary classifier for log-mel spectrograms (target vs non-target).
    Output: single sigmoid unit.
    Loss: Binary Focal Cross-Entropy (with safe fallbacks).
    """
    L2 = keras.regularizers.l2(l2)
    Conv = keras.layers.Conv2D

    inputs = keras.Input(shape=input_shape)

    # Block 1
    x = Conv(32, (3, 3), padding="same", kernel_regularizer=L2)(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = Conv(32, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D((2, 2))(x)   # 128x64 -> 64x32
    x = keras.layers.Dropout(dropout)(x)

    # Block 2
    x = Conv(64, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = Conv(64, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D((2, 2))(x)   # 64x32 -> 32x16
    x = keras.layers.Dropout(dropout)(x)

    # Block 3
    x = Conv(96, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = Conv(96, (3, 3), padding="same", kernel_regularizer=L2)(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D((2, 2))(x)   # 32x16 -> 16x8
    x = keras.layers.Dropout(dropout)(x)

    # Head
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(128, activation="relu", kernel_regularizer=L2)(x)
    x = keras.layers.Dropout(dropout)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)  # binary

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss=keras.losses.BinaryFocalCrossentropy(
            alpha=alpha,
            gamma=gamma,
            name='binary_focal_crossentropy'
        ),
        metrics=[
            keras.metrics.MeanSquaredError(name='Brier score'),
            keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
            keras.metrics.AUC(curve="ROC", name="auc"),
            keras.metrics.AUC(curve="PR",  name="auprc"),
            keras.metrics.Precision(name="precision", thresholds=0.5),
            keras.metrics.Recall(name="recall", thresholds=0.5),
        ],
    )
    return model


In [66]:
import math

def _map_pos(x, y):
    return audio_pipeline(x, augment=True), y

def _map_neg(x, y):
    return audio_pipeline(x, augment=True), y

def plan_epoch_counts(n_pos_train: int, neg_pos_ratio: float = 2.0) -> Tuple[int, int]:
    P_pos = int(n_pos_train)
    P_neg = int(round(neg_pos_ratio * P_pos))
    return P_pos, P_neg 

def sample_train_negatives(neg_all: Sequence[str], n_neg: int, seed: Optional[int] = None) -> Sequence[str]:
    n = min(n_neg, len(neg_all))
    rng = random.Random(seed)
    return rng.sample(list(neg_all), n) if n > 0 else []

def make_fixed_val_negatives(neg_all: Sequence[str], n_pos_val: int, neg_pos_ratio: int = NEG_POS_RATIO, seed: int = RANDOM_STATE) -> Sequence[str]:
    n_neg = min(neg_pos_ratio * n_pos_val, len(neg_all))
    rng = random.Random(seed)
    return rng.sample(list(neg_all), n_neg) if n_neg > 0 else []

STEPS_PER_TRAINING_EPOCH = 0
STEPS_PER_VALIDATION_EPOCH = 0

def build_train_dataset(pos_files: Sequence[str], neg_files: Sequence[str], batch_size: int = BATCH_SIZE, shuffle: bool = True) -> tf.data.Dataset:
    global STEPS_PER_TRAINING_EPOCH
    
    labels_pos = tf.ones([len(pos_files)], dtype=tf.float32)
    labels_neg = tf.zeros([len(neg_files)], dtype=tf.float32)
    
    ds_pos = tf.data.Dataset.from_tensor_slices((list(pos_files), labels_pos)).map(_map_pos, num_parallel_calls=tf.data.AUTOTUNE)
    ds_neg = tf.data.Dataset.from_tensor_slices((list(neg_files), labels_neg)).map(_map_neg, num_parallel_calls=tf.data.AUTOTUNE)

    # Set the amount of steps
    STEPS_PER_TRAINING_EPOCH = math.ceil((len(ds_pos) + len(ds_neg))/BATCH_SIZE)
    ds = ds_pos.concatenate(ds_neg)
    if shuffle:
        ds = ds.shuffle(buffer_size=len(pos_files) + len(neg_files), seed=RANDOM_STATE, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).cache().repeat()
    return ds


def build_val_dataset(pos_files: Sequence[str], neg_files_fixed: Sequence[str], batch_size: int = BATCH_SIZE) -> tf.data.Dataset:
    global STEPS_PER_VALIDATION_EPOCH

    y_pos = tf.ones([len(pos_files)],        dtype=tf.float32)
    y_neg = tf.zeros([len(neg_files_fixed)], dtype=tf.float32)

    ds_pos = tf.data.Dataset.from_tensor_slices((list(pos_files), y_pos)).map(lambda x,y: (audio_pipeline(x, augment=False), y), num_parallel_calls=tf.data.AUTOTUNE)
    ds_neg = tf.data.Dataset.from_tensor_slices((list(neg_files_fixed), y_neg)).map(lambda x,y: (audio_pipeline(x, augment=False), y), num_parallel_calls=tf.data.AUTOTUNE)

    STEPS_PER_VALIDATION_EPOCH = math.ceil((len(y_pos)) / BATCH_SIZE)

    return ds_pos.concatenate(ds_neg).batch(batch_size).cache()

def build_file_lists(df: pd.DataFrame, idx, target=TARGET):
    sub = df.iloc[idx]
    pos = sub[sub.primary_label == target]["path"].tolist()
    neg = sub[sub.primary_label != target]["path"].tolist()
    return pos, neg

def make_epoch_train_dataset(pos_tr_all: Sequence[str], neg_tr_all: Sequence[str],
                             neg_pos_ratio: float = 2.0, batch_size: int = BATCH_SIZE, seed: Optional[int] = RANDOM_STATE) -> tf.data.Dataset:
    P_pos, P_neg = plan_epoch_counts(len(pos_tr_all), neg_pos_ratio)
    neg_epoch = sample_train_negatives(neg_tr_all, P_neg, seed=seed)
    return build_train_dataset(pos_tr_all, neg_epoch, batch_size=batch_size, shuffle=True)

def make_fixed_val_dataset(pos_va_all: Sequence[str], neg_va_all: Sequence[str],
                           neg_pos_ratio: int = 3, batch_size: int = BATCH_SIZE, seed: int = RANDOM_STATE) -> tf.data.Dataset:
    neg_fixed = make_fixed_val_negatives(neg_va_all, len(pos_va_all), neg_pos_ratio=neg_pos_ratio, seed=seed)
    return build_val_dataset(pos_va_all, neg_fixed, batch_size=batch_size)


def build_all_file_lists(df: pd.DataFrame, folds: dict, epoch: int = 0):
    out = []
    for fold in folds:
        outer_fold = int(fold["outer_fold"])
        outer_train_idx = fold["outer_train_idx"]
        outer_test_idx = fold["outer_test_idx"]
    
        # Build the file lists for the outer fold
        pos_tr_all, neg_tr_all = build_file_lists(df, outer_train_idx)    
        pos_test_all, neg_test_all = build_file_lists(df, outer_test_idx)
    

        # Rotate negatives each epoch via seed that depends on (outer, inner, epoch)
        outer_train_ds = make_epoch_train_dataset(
            pos_tr_all,
            neg_tr_all,
            neg_pos_ratio=TRAIN_NEG_RATIO,
            batch_size=BATCH_SIZE,
            seed=outer_fold,
        )

        outer_test_ds = make_fixed_val_dataset(
            pos_test_all,
            neg_test_all,
            neg_pos_ratio=VAL_NEG_RATIO,
            batch_size=BATCH_SIZE,
            seed=outer_fold,  # fixed per outer fold
        )
    
        
        inner_list = []
        for inner in fold["inner_folds"]:
            inner_fold = int(inner["inner_fold"])
            inner_train_idx = inner["inner_train_idx"]
            inner_val_idx   = inner["inner_val_idx"]

            pos_tr_all, neg_tr_all = build_file_lists(df, inner_train_idx)
            pos_va_all, neg_va_all = build_file_lists(df, inner_val_idx)

            # Rotate negatives each epoch via seed that depends on (outer, inner, epoch)
            train_ds = make_epoch_train_dataset(
                pos_tr_all,
                neg_tr_all,
                neg_pos_ratio=TRAIN_NEG_RATIO,
                batch_size=BATCH_SIZE,
                seed=outer_fold * 100_000 + inner_fold * 1_000 + epoch,
            )

            # Fixed validation and test datasets (reproducible seeds per outer/inner)
            val_ds = make_fixed_val_dataset(
                pos_va_all,
                neg_va_all,
                neg_pos_ratio=VAL_NEG_RATIO,
                batch_size=BATCH_SIZE,
                seed=outer_fold * 100_000 + inner_fold,
            )

            test_ds = make_fixed_val_dataset(
                pos_test_all,
                neg_test_all,
                neg_pos_ratio=VAL_NEG_RATIO,
                batch_size=BATCH_SIZE,
                seed=outer_fold,  # fixed per outer fold
            )

            inner_list.append({
                "inner_fold": inner_fold,
                "train_ds": train_ds,
                "val_ds":   val_ds,
                "test_ds":  test_ds,
            })
        out.append({"outer_fold": outer_fold, "train_ds":outer_train_ds, "test_ds":outer_test_ds, "inner": inner_list})
    return out
        

datasets = build_all_file_lists(birdclef_df, cross_validation_sets)

In [67]:
per_fold_results = []

EPOCHS_PER_FOLD = 50

hparams_per_fold = {
    0: {"lr":1e-3, "gamma":1.0, "alpha":0.25},
    1: {"lr":1e-3, "gamma":2.0, "alpha":0.5},
    2: {"lr":1e-3, "gamma":3.0, "alpha":0.75},
    3: {"lr":1e-3, "gamma":4.0, "alpha":0.25},
    4: {"lr":1e-3, "gamma":5.0, "alpha":0.25},
}

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early = EarlyStopping(monitor="val_loss", mode="min", patience=8, min_delta=1e-3, restore_best_weights=True, verbose=1)
ckpt  = ModelCheckpoint("output/best.keras", monitor="val_loss", mode="min", save_best_only=True, verbose=1)


for outer_fold in datasets:
    print(f"[INFO]: Running outer fold: {outer_fold["outer_fold"]}")
    inner_scores = []
    for inner_fold in outer_fold["inner"]:
        print(f"[INFO]: Running inner fold: {inner_fold["inner_fold"]}")
        model = build_binary_cnn(input_shape=INPUT_SHAPE, 
                                 lr=hparams_per_fold[inner_fold["inner_fold"]]["lr"], 
                                 gamma=hparams_per_fold[inner_fold["inner_fold"]]["gamma"], 
                                 alpha=hparams_per_fold[inner_fold["inner_fold"]]["alpha"])

        model.fit(
            inner_fold["train_ds"],
            epochs = EPOCHS_PER_FOLD,
            validation_data=inner_fold["val_ds"],
            steps_per_epoch=STEPS_PER_TRAINING_EPOCH,
            validation_steps = STEPS_PER_VALIDATION_EPOCH,
            verbose=1,
            callbacks=[early, ckpt]
        )
        
        vals = model.evaluate(inner_fold["test_ds"], verbose=0)
        print(model.metrics_names)
        print(vals)
        inner_scores.append(vals[2])
    
    mean_test_acc = np.mean(inner_scores)
    print(f"Inner CV mean test acc: {mean_test_acc:.3f}")
    
    # Retrain with the best parameters
    best_inner = np.argmax(inner_scores)
    
    model = build_binary_cnn(input_shape=INPUT_SHAPE, 
                                lr=hparams_per_fold[best_inner]["lr"], 
                                gamma=hparams_per_fold[best_inner]["gamma"], 
                                alpha=hparams_per_fold[best_inner]["alpha"])
    model.fit(
        outer_fold["train_ds"],
        epochs = EPOCHS_PER_FOLD,
        steps_per_epoch=STEPS_PER_TRAINING_EPOCH,
        validation_steps = STEPS_PER_VALIDATION_EPOCH,
        verbose=1,
        callbacks=[early, ckpt]
    )
    
    vals = model.evaluate(outer_fold["test_ds"], verbose=0)
    per_fold_results.append(vals[2])

print(per_fold_results)

[INFO]: Running outer fold: 1
[INFO]: Running inner fold: 1
Epoch 1/50
1/2 ━━━━━━━━━━━━━━━━━━━━ 9s 10s/step - Brier score: 0.1948 - acc: 0.7500 - auc: 0.4714 - auprc: 0.2249 - loss: 0.2231 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 1: val_loss improved from None to 0.48267, saving model to output/best.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 4s/step - Brier score: 0.2227 - acc: 0.6562 - auc: 0.5419 - auprc: 0.2831 - loss: 0.2380 - precision: 0.3333 - recall: 0.2941 - val_Brier score: 0.2896 - val_acc: 0.5833 - val_auc: 0.5254 - val_auprc: 0.4438 - val_loss: 0.4827 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - Brier score: 0.2263 - acc: 0.6094 - auc: 0.5698 - auprc: 0.4219 - loss: 0.2315 - precision: 0.3333 - recall: 0.1711
Epoch 2: val_loss did not improve from 0.48267
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - Brier score: 0.2420 - acc: 0.5625 - auc: 0.5431 - auprc: 0.4308 - loss: 0.2603 - precision: 0.3333 - recall: 0.1200 - 

/home/joris/.pyenv/versions/3.12.11/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: Brier score,acc,auc,auprc,loss,precision,recall
  current = self.get_monitor_value(logs)
/home/joris/.pyenv/versions/3.12.11/lib/python3.12/site-packages/keras/src/callbacks/model_checkpoint.py:276: UserWarning: Can save best model only with val_loss available.
  if self._should_save_model(epoch, batch, logs, filepath):


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - Brier score: 0.2567 - acc: 0.5938 - auc: 0.4662 - auprc: 0.2450 - loss: 0.4469 - precision: 0.3043 - recall: 0.4118
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - Brier score: 0.2597 - acc: 0.6094 - auc: 0.3949 - auprc: 0.2975 - loss: 0.5142 - precision: 0.2222 - recall: 0.1000
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - Brier score: 0.2572 - acc: 0.5667 - auc: 0.5347 - auprc: 0.3904 - loss: 0.4697 - precision: 0.2500 - recall: 0.0417         
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - Brier score: 0.2329 - acc: 0.6094 - auc: 0.5514 - auprc: 0.4465 - loss: 0.3872 - precision: 0.4286 - recall: 0.2609
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - Brier score: 0.2755 - acc: 0.4531 - auc: 0.3411 - auprc: 0.2096 - loss: 0.4578 - precision: 0.1786 - recall: 0.2941
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - Brier score: 0.2446 - acc: 0.6250 - auc: 0.5767 - auprc: 0.3449 - loss: 0.3972 - precision: 0.3750 - recall: 0.30

/home/joris/.pyenv/versions/3.12.11/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - Brier score: 0.2421 - acc: 0.6797 - auc: 0.4593 - auprc: 0.2465 - loss: 0.2685 - precision: 0.3452 - recall: 0.2721 
Epoch 2: val_loss did not improve from 0.48267
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - Brier score: 0.2280 - acc: 0.6719 - auc: 0.5307 - auprc: 0.2863 - loss: 0.2468 - precision: 0.3571 - recall: 0.2941 - val_Brier score: 0.4275 - val_acc: 0.3333 - val_auc: 0.4738 - val_auprc: 0.3020 - val_loss: 0.6956 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 3/50
1/2 ━━━━━━━━━━━━━━━━━━━━ 3s 4s/step - Brier score: 0.2748 - acc: 0.5357 - auc: 0.4250 - auprc: 0.3115 - loss: 0.3598 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3: val_loss improved from 0.48267 to 0.25331, saving model to output/best.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 174ms/step - Brier score: 0.2551 - acc: 0.5667 - auc: 0.4856 - auprc: 0.3638 - loss: 0.3196 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_Brier score: 0.2750 - val_acc: 0.3333 - val_auc: 0.4699 - val_a

2025-09-02 20:59:48.194136: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.



Epoch 2: val_loss did not improve from 0.13739
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - Brier score: 0.2405 - acc: 0.6296 - auc: 0.5556 - auprc: 0.3569 - loss: 0.1924 - precision: 0.4286 - recall: 0.3333 - val_Brier score: 0.3441 - val_acc: 0.5417 - val_auc: 0.5789 - val_auprc: 0.6283 - val_loss: 0.6111 - val_precision: 0.5417 - val_recall: 1.0000
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - Brier score: 0.2169 - acc: 0.5341 - auc: 0.6405 - auprc: 0.4362 - loss: 0.1250 - precision: 0.2159 - recall: 0.1465 
Epoch 3: val_loss did not improve from 0.13739
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step - Brier score: 0.2219 - acc: 0.5370 - auc: 0.6057 - auprc: 0.4027 - loss: 0.1313 - precision: 0.1818 - recall: 0.1111 - val_Brier score: 0.3307 - val_acc: 0.5417 - val_auc: 0.5846 - val_auprc: 0.6323 - val_loss: 0.5316 - val_precision: 0.5417 - val_recall: 1.0000
Epoch 4/50
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - Brier score: 0.2373 - acc: 0.5938 - auc: 0.6082 - auprc: 0.5024 - loss: 0.1385 - pr

In [68]:
# Now we can train the actual model we use for the soundscapes based on the best configuration

model = build_binary_cnn(input_shape=INPUT_SHAPE, 
                         lr=hparams_per_fold[best_inner]["lr"], 
                         gamma=hparams_per_fold[best_inner]["gamma"], 
                         alpha=hparams_per_fold[best_inner]["alpha"])
model.fit(
    datasets[np.argmax(per_fold_results)]["train_ds"],
    epochs = EPOCHS_PER_FOLD,
    steps_per_epoch=STEPS_PER_TRAINING_EPOCH,
    callbacks=[early, ckpt]
)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 114ms/step - Brier score: 0.2424 - acc: 0.5781 - auc: 0.6417 - auprc: 0.4242 - loss: 0.4188 - precision: 0.4211 - recall: 0.7619
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - Brier score: 0.2620 - acc: 0.5938 - auc: 0.5172 - auprc: 0.3610 - loss: 0.5261 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - Brier score: 0.2539 - acc: 0.6094 - auc: 0.3793 - auprc: 0.2667 - loss: 0.4524 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 204ms/step - Brier score: 0.2440 - acc: 0.6000 - auc: 0.5248 - auprc: 0.3871 - loss: 0.4013 - precision: 0.3889 - recall: 0.4375
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - Brier score: 0.2277 - acc: 0.6250 - auc: 0.5835 - auprc: 0.3817 - loss: 0.3708 - precision: 0.4000 - recall: 0.4000
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - Brier score: 0.2372 - acc: 0.6719 - auc: 0.5959 - auprc: 0.4766 - loss: 0.3975 - precision: 

In [ ]:
import math
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.metrics import precision_score, recall_score, f1_score


SOUNDSCAPE_DIR = "datasets/birdclef_2021/train_soundscapes"

# Load the labels
soundscape_labels = pd.read_csv("datasets/birdclef_2021/train_soundscape_labels.csv")

EXP_NAME = "mixup"
OUTPUT_DIR = "outputs"
EXP_DIR = os.path.join(OUTPUT_DIR, EXP_NAME)

# Output folders
CM_DIR = os.path.join(EXP_DIR, "confusion_matrices")
METRICS_DIR = os.path.join(EXP_DIR, "metrics")
SCAN_DIR = os.path.join(EXP_DIR, "threshold_scans")
os.makedirs(EXP_DIR, exist_ok=True)
os.makedirs(CM_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)
os.makedirs(SCAN_DIR, exist_ok=True)

soundscapes = []
def process_soundscape(path: str):
    # Load the soundscape
    y, _ = librosa.load(os.path.join(SOUNDSCAPE_DIR, path), sr=SAMPLE_RATE)

    # Filter for the current file
    site = path.split("_")[1]
    audio_id = int(path.split("_")[0])

    # Pass it through a bandpass
    b, a = signal.butter(4, [200, 7999], fs=SAMPLE_RATE, btype='band')
    y = signal.lfilter(b, a, y)

    chunks = []
    for i in range(0, math.ceil(len(y)/BYTES_PER_SECOND)):
        # Cut it into chunks of BYTES_PER_SECOND
        chunk = y[i*BYTES_PER_SECOND:(i+1)*BYTES_PER_SECOND]
        
        # Convert to spectrogram
        spec = generate_mel_spectrogram(chunk)
        chunks.append(spec)
        
    labels = soundscape_labels[(soundscape_labels["site"] == site) & (soundscape_labels["audio_id"] == audio_id)].copy()
    labels["class"] = labels["birds"].apply(lambda x: "target" if TARGET in x else "other")
    
    # Predict the labels with the model
    predictions = model.predict(np.array(chunks), verbose=0)
    labels["predictions"] = predictions
    
    soundscapes.append({
        "site":site,
        "audio_id":audio_id,
        "labels":labels
    })
    
    # Also export the DF
    labels.to_csv(os.path.join(EXP_DIR, f"{site}_{audio_id}.csv"))
    
    # Clean up
    del y, chunks

for f in os.listdir(SOUNDSCAPE_DIR):
    process_soundscape(f)

In [70]:
# Results processing and comparison

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score

# Ensure output dirs exist (adjust paths if you defined them earlier)
CM_DIR = os.path.join(EXP_DIR, "confusion_matrices")
METRICS_DIR = os.path.join(EXP_DIR, "metrics")
SCAN_DIR = os.path.join(EXP_DIR, "threshold_scans")
os.makedirs(CM_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)
os.makedirs(SCAN_DIR, exist_ok=True)

def plot_confusion_matrix(y_true, y_pred, labels=None, class_names=None,
                          normalize=False, cmap="Blues"):
    """
    Plot a confusion matrix with fixed shape using all known labels.
    """
    if labels is None:
        labels = np.unique(np.concatenate([y_true, y_pred]))
    if class_names is None:
        class_names = [str(l) for l in labels]

    cm = confusion_matrix(y_true, y_pred, labels=labels)

    if normalize:
        cm = cm.astype(np.float32) / cm.sum(axis=1, keepdims=True)
        cm = np.nan_to_num(cm)

    fig, ax = plt.subplots(figsize=(5, 5))
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)

    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel("Counts" if not normalize else "Proportion", rotation=-90, va="bottom")

    ax.set(
        xticks=np.arange(len(labels)),
        yticks=np.arange(len(labels)),
        xticklabels=class_names,
        yticklabels=class_names,
        ylabel="True label",
        xlabel="Predicted label"
    )
    plt.setp(ax.get_xticklabels(), rotation=0)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0 if cm.size else 0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    fig.tight_layout()
    return ax

def find_threshold_macroF1(df: pd.DataFrame, step=0.01, eps=1e-3):
    label_col = "label" if "label" in df.columns else "class"
    y_true_str = df[label_col].values
    y_prob = df["predictions"].astype(float).values
    labels = ["target","other"]

    thresholds = np.arange(eps, 1.0 + 1e-9, step)
    rows = []
    for t in thresholds:
        y_pred_str = np.where(y_prob >= t, "target", "other")
        cm = confusion_matrix(y_true_str, y_pred_str, labels=labels)
        # F1 for 'target'
        tp_t, fp_t, fn_t = int(cm[0,0]), int(cm[1,0]), int(cm[0,1])
        p_t = tp_t / (tp_t + fp_t) if (tp_t + fp_t) else 0.0
        r_t = tp_t / (tp_t + fn_t) if (tp_t + fn_t) else 0.0
        f1_t = 2*p_t*r_t/(p_t+r_t) if (p_t+r_t) else 0.0
        # F1 for 'other' (as positive)
        tp_o, fp_o, fn_o = int(cm[1,1]), int(cm[0,1]), int(cm[1,0])
        p_o = tp_o / (tp_o + fp_o) if (tp_o + fp_o) else 0.0
        r_o = tp_o / (tp_o + fn_o) if (tp_o + fn_o) else 0.0
        f1_o = 2*p_o*r_o/(p_o+r_o) if (p_o+r_o) else 0.0
        rows.append({"threshold": t, "f1_macro": (f1_t + f1_o)/2.0})

    dfm = pd.DataFrame(rows)
    best_row = dfm.iloc[dfm["f1_macro"].values.argmax()]
    return float(best_row["threshold"]), dfm

def save_current_fig(fig, path_png, path_svg=None, dpi=200):
    fig.savefig(path_png, dpi=dpi, bbox_inches="tight")
    if path_svg:
        fig.savefig(path_svg, bbox_inches="tight")

# Safe division P/R/F1 helper (inline, no external deps)
def _prf(tp, fp, fn):
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    rec  = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1   = (2 * prec * rec) / (prec + rec) if (prec + rec) > 0 else 0.0
    return prec, rec, f1

all_rows = []

# choose which TARGET metric to optimize: "f1" | "precision" | "recall"
TARGET_OPTIMIZE = "f1"

for rslt in soundscapes:
    df = rslt["labels"]

    base = f"{rslt['site']}_{rslt['audio_id']}"

    # ---- Optimize threshold for the TARGET class on THIS file ----
    y_true_bin = (df["class"].values == "target").astype(int)
    y_prob = df["predictions"].astype(float).values

    thresholds = np.arange(0.01, 1.0 + 1e-9, 0.01)  # exclude 0.0 to prevent degenerate all-target
    rows = []
    for t in thresholds:
        y_pred_bin = (y_prob >= t).astype(int)
        prec = precision_score(y_true_bin, y_pred_bin, zero_division=0)
        rec  = recall_score(y_true_bin, y_pred_bin, zero_division=0)
        f1   = f1_score(y_true_bin, y_pred_bin, zero_division=0)
        rows.append({"threshold": float(t), "precision": prec, "recall": rec, "f1": f1})

    table = pd.DataFrame(rows)
    best_idx = table[TARGET_OPTIMIZE].values.argmax()
    best_row = table.iloc[best_idx]
    best_thr = float(best_row["threshold"])
    best_metrics = {"precision": float(best_row["precision"]),
                    "recall": float(best_row["recall"]),
                    "f1": float(best_row["f1"])}

    print(f"[{base}] Best (target-{TARGET_OPTIMIZE}) threshold: {best_thr:.2f} | "
          f"P={best_metrics['precision']:.3f} R={best_metrics['recall']:.3f} F1={best_metrics['f1']:.3f}")

    # Save the per-file threshold scan (optional but handy to inspect)
    table.to_csv(os.path.join(SCAN_DIR, f"{base}_threshold_scan.csv"), index=False)

    # Apply best threshold to produce hard labels
    df["predicted_class"] = np.where(df["predictions"] >= best_thr, "target", "other")

    # Fixed label order
    labels_fixed = ["target", "other"]

    # Confusion matrix and save plot
    cm = confusion_matrix(df["class"], df["predicted_class"], labels=labels_fixed)
    ax = plot_confusion_matrix(df["class"], df["predicted_class"], labels=labels_fixed, class_names=labels_fixed)
    fig = ax.figure
    save_current_fig(
        fig,
        os.path.join(CM_DIR, f"{base}_cm.png"),
        path_svg=os.path.join(CM_DIR, f"{base}_cm.svg"),
        dpi=220
    )
    plt.close(fig)

    # --- Single consolidated metrics CSV per soundscape (per-class + global) ---
    # CM layout: rows true [target, other], cols pred [target, other]
    tp_t, fp_t, fn_t, tn_t = int(cm[0, 0]), int(cm[1, 0]), int(cm[0, 1]), int(cm[1, 1])  # positive='target'
    tp_o, fp_o, fn_o, tn_o = int(cm[1, 1]), int(cm[0, 1]), int(cm[1, 0]), int(cm[0, 0])  # positive='other'

    support_t = int((df["class"].values == "target").sum())
    support_o = int((df["class"].values == "other").sum())
    total = support_t + support_o

    p_t = tp_t / (tp_t + fp_t) if (tp_t + fp_t) > 0 else 0.0
    r_t = tp_t / (tp_t + fn_t) if (tp_t + fn_t) > 0 else 0.0
    f1_t = (2 * p_t * r_t) / (p_t + r_t) if (p_t + r_t) > 0 else 0.0

    p_o = tp_o / (tp_o + fp_o) if (tp_o + fp_o) > 0 else 0.0
    r_o = tp_o / (tp_o + fn_o) if (tp_o + fn_o) > 0 else 0.0
    f1_o = (2 * p_o * r_o) / (p_o + r_o) if (p_o + r_o) > 0 else 0.0

    acc = (tp_t + tn_t) / max(total, 1)

    micro_p = precision_score(y_true_bin, (df["predicted_class"].values == "target").astype(int), zero_division=0)
    micro_r = recall_score(y_true_bin, (df["predicted_class"].values == "target").astype(int), zero_division=0)
    micro_f1 = f1_score(y_true_bin, (df["predicted_class"].values == "target").astype(int), zero_division=0)

    macro_p = (p_t + p_o) / 2.0
    macro_r = (r_t + r_o) / 2.0
    macro_f1 = (f1_t + f1_o) / 2.0

    metrics_rows = [
        {
            "site": rslt["site"], "audio_id": rslt["audio_id"], "class": "target",
            "threshold": best_thr, "support": support_t,
            "tp": tp_t, "fp": fp_t, "fn": fn_t, "tn": tn_t,
            "precision": p_t, "recall": r_t, "f1": f1_t, "accuracy": acc
        },
        {
            "site": rslt["site"], "audio_id": rslt["audio_id"], "class": "other",
            "threshold": best_thr, "support": support_o,
            "tp": tp_o, "fp": fp_o, "fn": fn_o, "tn": tn_o,
            "precision": p_o, "recall": r_o, "f1": f1_o, "accuracy": acc
        },
        {
            "site": rslt["site"], "audio_id": rslt["audio_id"], "class": "global",
            "threshold": best_thr, "support": total,
            "tp": tp_t + tp_o, "fp": fp_t + fp_o, "fn": fn_t + fn_o, "tn": tn_t + tn_o,
            "precision_macro": macro_p, "recall_macro": macro_r, "f1_macro": macro_f1,
            "precision_micro": micro_p, "recall_micro": micro_r, "f1_micro": micro_f1,
            "accuracy": acc
        },
    ]
    pd.DataFrame(metrics_rows).to_csv(os.path.join(METRICS_DIR, f"{base}_metrics.csv"), index=False)

    # Optional: text report
    report_txt = classification_report(df["class"], df["predicted_class"], labels=labels_fixed, zero_division=0)
    with open(os.path.join(METRICS_DIR, f"{base}_classification_report.txt"), "w") as f:
        f.write(f"Best threshold (target-{TARGET_OPTIMIZE}): {best_thr:.4f}\n\n")
        f.write(report_txt)

    # Accumulate for global summary if you still need it later
    all_rows.append(df[["class", "predictions"]].assign(site=rslt["site"], audio_id=rslt["audio_id"]))

# ------- Global summary across all soundscapes (optimize TARGET metrics) -------
if all_rows:
    # Build a single DataFrame across all soundscapes with site/audio_id attached
    df_all = pd.concat(
        [s["labels"].assign(site=s["site"], audio_id=s["audio_id"]) for s in soundscapes],
        ignore_index=True
    )

    # Ground-truth column can be 'class' (recommended) or 'label'
    gt_col = "class" if "class" in df_all.columns else "label"
    assert gt_col in df_all.columns and "predictions" in df_all.columns, \
        "Expected columns for ground truth ('class' or 'label') and 'predictions'."

    # -------- Optimize global threshold for TARGET-class metric --------
    # Only use files that contain at least one 'target' to PICK the threshold
    files_with_pos = (
        df_all.groupby(["site", "audio_id"])[gt_col]
        .apply(lambda s: (s == "target").any())
    )
    pos_file_index = files_with_pos[files_with_pos].index
    mask_with_pos = df_all.set_index(["site", "audio_id"]).index.isin(pos_file_index)
    df_for_thr = df_all[mask_with_pos].reset_index(drop=True)

    # Choose which target metric to optimize: 'f1' | 'precision' | 'recall'
    TARGET_OPTIMIZE = "f1"

    if len(df_for_thr) > 0:
        y_true = (df_for_thr[gt_col].values == "target").astype(int)
        y_prob = df_for_thr["predictions"].astype(float).values

        thresholds = np.arange(0.01, 1.0 + 1e-9, 0.01)  # exclude 0 to avoid "all-target" degenerate choice
        best_thr, best_val = 0.5, -1.0

        for t in thresholds:
            y_pred = (y_prob >= t).astype(int)
            if TARGET_OPTIMIZE == "precision":
                val = precision_score(y_true, y_pred, zero_division=0)
            elif TARGET_OPTIMIZE == "recall":
                val = recall_score(y_true, y_pred, zero_division=0)
            else:  # "f1"
                val = f1_score(y_true, y_pred, zero_division=0)
            if val > best_val:
                best_val, best_thr = val, float(t)
        g_thr = best_thr
    else:
        g_thr = 0.5  # fallback if no positives anywhere
    print(f"[GLOBAL] Using threshold {g_thr:.2f} optimized for target {TARGET_OPTIMIZE.upper()}")

    # ---- Apply global threshold to ALL rows (including files with no target) ----
    y_true_all = df_all[gt_col].values
    y_pred_all = np.where(df_all["predictions"].values >= g_thr, "target", "other")
    labels_fixed = ["target", "other"]

    # Plot/save global CM
    cm_all = confusion_matrix(y_true_all, y_pred_all, labels=labels_fixed)
    ax = plot_confusion_matrix(y_true_all, y_pred_all, labels=labels_fixed, class_names=labels_fixed)
    fig = ax.figure
    save_current_fig(
        fig,
        os.path.join(CM_DIR, "GLOBAL_cm.png"),
        path_svg=os.path.join(CM_DIR, "GLOBAL_cm.svg"),
        dpi=220
    )
    plt.close(fig)

    # Components (rows true [target, other], cols pred [target, other])
    tp_t, fp_t, fn_t, tn_t = int(cm_all[0, 0]), int(cm_all[1, 0]), int(cm_all[0, 1]), int(cm_all[1, 1])
    tp_o, fp_o, fn_o, tn_o = int(cm_all[1, 1]), int(cm_all[0, 1]), int(cm_all[1, 0]), int(cm_all[0, 0])

    support_t = int((y_true_all == "target").sum())
    support_o = int((y_true_all == "other").sum())
    total = support_t + support_o

    def _prf(tp, fp, fn):
        prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        rec  = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1   = (2 * prec * rec) / (prec + rec) if (prec + rec) > 0 else 0.0
        return prec, rec, f1

    p_t, r_t, f1_t = _prf(tp_t, fp_t, fn_t)
    p_o, r_o, f1_o = _prf(tp_o, fp_o, fn_o)

    acc = (tp_t + tn_t) / max(total, 1)

    # Micro and macro (binary)
    y_true_bin = (y_true_all == "target").astype(int)
    y_pred_bin = (y_pred_all == "target").astype(int)
    micro_p = precision_score(y_true_bin, y_pred_bin, zero_division=0)
    micro_r = recall_score(y_true_bin, y_pred_bin, zero_division=0)
    micro_f1 = f1_score(y_true_bin, y_pred_bin, zero_division=0)

    macro_p = (p_t + p_o) / 2.0
    macro_r = (r_t + r_o) / 2.0
    macro_f1 = (f1_t + f1_o) / 2.0

    # Save a single GLOBAL metrics CSV with per-class + global rows
    global_rows = [
        {
            "site": "GLOBAL", "audio_id": -1, "class": "target",
            "threshold": g_thr, "support": support_t,
            "tp": tp_t, "fp": fp_t, "fn": fn_t, "tn": tn_t,
            "precision": p_t, "recall": r_t, "f1": f1_t, "accuracy": acc
        },
        {
            "site": "GLOBAL", "audio_id": -1, "class": "other",
            "threshold": g_thr, "support": support_o,
            "tp": tp_o, "fp": fp_o, "fn": fn_o, "tn": tn_o,
            "precision": p_o, "recall": r_o, "f1": f1_o, "accuracy": acc
        },
        {
            "site": "GLOBAL", "audio_id": -1, "class": "global",
            "threshold": g_thr, "support": total,
            "tp": tp_t + tp_o, "fp": fp_t + fp_o, "fn": fn_t + fn_o, "tn": tn_t + tn_o,
            "precision_macro": macro_p, "recall_macro": macro_r, "f1_macro": macro_f1,
            "precision_micro": micro_p, "recall_micro": micro_r, "f1_micro": micro_f1,
            "accuracy": acc
        },
    ]
    pd.DataFrame(global_rows).to_csv(os.path.join(METRICS_DIR, "GLOBAL_metrics.csv"), index=False)

    with open(os.path.join(METRICS_DIR, "GLOBAL_classification_report.txt"), "w") as f:
        f.write(f"Best threshold (target-{TARGET_OPTIMIZE}): {g_thr:.4f}\n\n")
        f.write(classification_report(y_true_all, y_pred_all, labels=labels_fixed, zero_division=0))


[SSW_2782] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[SSW_7843] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[COR_7954] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[COR_31928] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[SSW_42907] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[SSW_28933] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[COR_50878] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[SSW_10534] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[COR_57610] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[SSW_26709] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[SSW_20152] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[COR_26746] Best (target-f1) threshold: 0.17 | P=0.207 R=0.783 F1=0.327
[COR_44957] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0.000
[SSW_14473] Best (target-f1) threshold: 0.01 | P=0.000 R=0.000 F1=0